# Lección 9: Modelos Avanzados – Panel Data, Efectos Mixtos, Series de Tiempo y GMM

En esta lección cubriremos cuatro técnicas avanzadas para análisis de datos longitudinales y de series, así como estimación mediante métodos de momentos generales.

## Datos de Panel: Efectos Fijos y Aleatorios

a. Preparar datos de panel

En esencial que tu `DataFrame` tenga un índice multiple `(entity, time)` o columnas que lo identifiquen.

In [3]:
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
import statsmodels.api as sm

# Ejemplo sintético
df = pd.DataFrame({
    "firm":    ["A","A","B","B","C","C"],
    "year":    [2018,2019,2018,2019,2018,2019],
    "y":       [2.1,2.3,1.9,2.0,3.2,3.5],
    "x1":      [5.0,5.1,4.8,4.9,6.0,6.1],
})

df = df.set_index(["firm", "year"])

b. Modelo de Efectos Fijos

In [4]:
# Agregar constante
exog = sm.add_constant(df[["x1"]])
mod_fe = PanelOLS(df["y"], exog, entity_effects=True)
res_fe = mod_fe.fit()
print(res_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                      y   R-squared:                        0.8571
Estimator:                   PanelOLS   R-squared (Between):              0.4783
No. Observations:                   6   R-squared (Within):               0.8571
Date:                Sun, Jun 15 2025   R-squared (Overall):              0.4899
Time:                        10:30:26   Log-likelihood                    10.677
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      12.000
Entities:                           3   P-value                           0.0742
Avg Obs:                       2.0000   Distribution:                     F(1,2)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             12.000
                            

* `entity_effects=True` añade interceptos propios para cada firma.
* controla heterogeneidad inobservable constante en el tiempo.

c. Modelo de Efectos Aleatorios

In [6]:
mod_re = RandomEffects(df["y"], exog)
res_re = mod_re.fit()
print(res_re.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:                      y   R-squared:                        0.9910
Estimator:              RandomEffects   R-squared (Between):              0.9998
No. Observations:                   6   R-squared (Within):               0.7091
Date:                Sun, Jun 15 2025   R-squared (Overall):              0.9910
Time:                        10:32:09   Log-likelihood                    8.4831
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      438.75
Entities:                           3   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                     F(1,4)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             438.75
                            

* Utiliza un término aleatorio en lugar de interceptos fijos.
* Puedes contrastar con Hausman test para elegir entre FE y RE.

## Modelos de Efectos Mixtos (Mixed Models)

Usa `MixedLM` de statsmodels para incluir efectos fijos y aletorios en jerarquías más complejas.

In [11]:
import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLM

# Supongamos datos de estudiantes en escuelas
data = pd.DataFrame({
    "school":   ["S1"]*5 + ["S2"]*5,
    "student":  list(range(10)),
    "score":    [75,80,78,82,79,88,85,87,90,89],
    "hours":    [2,3,2.5,3.5,3,4,4.5,4,5,4.5]
})

# Efecto aleatorio por escuela
model_mixed = MixedLM.from_formula("score ~ hours", groups = "school", data = data)
res_mixed = model_mixed.fit()
print(res_mixed.summary())

        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: score   
No. Observations: 10      Method:             REML    
No. Groups:       2       Scale:              2.2400  
Min. group size:  5       Log-Likelihood:     -17.3058
Max. group size:  5       Converged:          Yes     
Mean group size:  5.0                                 
------------------------------------------------------
            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------
Intercept   68.349    4.639 14.734 0.000 59.257 77.441
hours        4.153    1.230  3.376 0.001  1.742  6.564
school Var   3.367    5.302                           



* `groups` determina el clustering para los efectos aleatorios.
* Puedes añadir efectos aleatorios adicionales con `re_formula`.


## Series de Tiempo: Modelos ARIMA y Descomposición

a. Descomposición de series

```python
from statsmodels.tsa.seasonal import seasonal_decompose

serie = pd.Series(..., index=pd.date_range("2020-01-01", periods=36, freq="M"))
descomp = seasonal_decompose(serie, model="additive")
descomp.plot()
```
* `trend`, `seasonal` y `resid` para diagnóstico visual.

b. ARIMA

```python
from statsmodels.tsa.arima_model import ARIMA

# Ajustar un ARIMA(p,d,q)
model_arima = ARIMA(serie, order = (1,1,1))
res_arima = model_arima.fit()
print(res_arima.summary())

# Pronóstico
pred = res_arima.get_forecast(steps = 12)
pred_ci = pred.conf_int()
```

* `order=(p,d,q)` controla AR, diferenciación y M.
* Revisa AIC/BIC para seleccionar el mejor modelo.

## Estimación por GMM (Generalized Method of Moments)

La implementación en `statsmodels` se encuentra en el sandbox, o bien puedes usar `linearmodels.iv.GMM`.

In [14]:
from statsmodels.sandbox.regression.gmm import GMM

class LinearGMM(GMM):
    def momcond(self, params):
        beta = params
        X = self.exog
        y = self.endog
        u = y - X.dot(beta)
        # Condiciones: E[z * u] = 0, con z = X (exógenos puros)
        return (self.instrument * u[:, None])

# Datos
y = df["y"].values
X = exog.values
instr = X.copy()  # IV = X, para ilustración

gmm_mod = LinearGMM(y, X, instr)
res_gmm = gmm_mod.fit(start_params=[0,1], maxiter=2)
print(res_gmm.params)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
[-3.71428571  1.16883117]
